# Asset Selling - Optimal Stopping

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/pedronahum/stochastic-optimization/blob/master/notebooks/asset_selling.ipynb)

In [ ]:
!pip install -q jax jaxlib jaxtyping chex matplotlib
import os
if 'COLAB_GPU' in os.environ or not os.path.exists('problems'):
    !git clone https://github.com/pedronahum/stochastic-optimization.git
    os.chdir('stochastic-optimization')

In [ ]:
import jax
import jax.numpy as jnp
import matplotlib.pyplot as plt
import numpy as np
from problems.asset_selling import AssetSellingConfig, AssetSellingModel

In [ ]:
config = AssetSellingConfig(n_assets=3, horizon=100)
model = AssetSellingModel(config)
key = jax.random.PRNGKey(42)
state = model.init_state(key)
print('✓ Asset selling model ready')

In [ ]:
# Track asset prices and selling decisions
prices, decisions, rewards = [], [], []
for t in range(100):
    key, k1, k2 = jax.random.split(key, 3)
    decision = jnp.array([1.0, 0.0, 0.0])  # Sell first asset
    exog = model.sample_exogenous(k2, state, t)
    reward = model.reward(state, decision, exog)
    prices.append(state[:3].tolist())
    decisions.append(decision.tolist())
    rewards.append(float(reward))
    state = model.transition(state, decision, exog)

plt.figure(figsize=(12, 4))
plt.subplot(1, 2, 1)
for i in range(3):
    plt.plot([p[i] for p in prices], label=f'Asset {i+1}')
plt.title('Asset Prices')
plt.legend()
plt.subplot(1, 2, 2)
plt.plot(np.cumsum(rewards))
plt.title('Cumulative Revenue')
plt.tight_layout()
plt.show()